# pytorch customDataset

In [1]:
from torch.utils.data.dataset import Dataset
from glob import glob
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms as transforms
import numpy as np

class customDataset(Dataset):
    def __init__(self, transform):
        # --------------------------------------------
        # Initialize paths, transforms, and so on
        # --------------------------------------------
#         self.path = glob("D:\WorkSpace\jupyter\DataSet\SCUT-FBP5500_v2\Images\*.jpg")
        df = pd.read_excel("D:\WorkSpace\jupyter\DataSet\SCUT-FBP5500_v2\All_Ratings.xlsx", sheet_name="ALL")        
        self.fns = list(df.Filename)
        self.labels = list(df.Rating)
        self.transform = transform
        
    def __getitem__(self, index):
        # --------------------------------------------
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        # --------------------------------------------
        imgpath = "D:\\WorkSpace\\jupyter\\DataSet\\SCUT-FBP5500_v2\\Images\\%s" % (self.fns[index])
        img = Image.open(imgpath).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        label = torch.from_numpy(np.array([self.labels[index]-1], dtype="int32")).long().squeeze()
        return img, label
        
    def __len__(self):
        # --------------------------------------------
        # Indicate the total size of the dataset
        # --------------------------------------------
        return len(self.fns)
    
# Create train/valid transforms
transform = transforms.Compose([
    transforms.Resize((224,224), interpolation=3),
#     transforms.RandomCrop((224, 224)),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])
])
# valid_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\Program\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import time
import numpy as np
from torchvision import models
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

full_dataset = customDataset(transform = transform)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
num_classes = 6

In [3]:
# #输出第0张图片的大小
print(train_dataset[0][0].size)

<built-in method size of Tensor object at 0x000002679D200438>


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# print(torch.cuda.get_device_name(device))

cuda:0


# 創建數據加載器

In [5]:
from torch.utils.data import DataLoader

torch.manual_seed(1)
BATCH_SIZE = 2

train_loader = DataLoader(
    dataset=train_dataset,# TensorDataset类型数据集
    batch_size=BATCH_SIZE,# mini batch size
    shuffle=True# 设置随机洗牌
)

test_loader = DataLoader(
    dataset=test_dataset,# TensorDataset类型数据集
    batch_size=BATCH_SIZE,# mini batch size
    shuffle=True# 设置随机洗牌
)

In [6]:
batch = next(iter(train_loader))

In [7]:
print(batch[0].size())
print(batch[1].size())

torch.Size([2, 3, 224, 224])
torch.Size([2])


In [8]:
print(batch[1].squeeze())

tensor([2, 1])


In [9]:
print(len(train_loader))
print(len(test_loader))

132000
33000


# 構建網絡結構

In [10]:
# 两层卷积
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 使用序列工具快速构建
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(56 * 56 * 32, 5)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0), -1)  # reshape
        out = self.fc(out)
        return out

In [11]:
# 3x3 卷积
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)


# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=2):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(128 * 128, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [12]:
# 加載預訓練模型
model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features

# 把resnet的全连接层fc 替换成自己设置的线性层nn.Linear
# 输入维度是resnet.fc.in_features, 输出是196维
model_ft.fc = nn.Linear(num_ftrs, num_classes)

In [13]:
from torchvision import models
# cnn = CNN()
# cnn = ResNet(ResidualBlock, [2, 2, 2, 2])
cnn = model_ft
print(cnn)
if torch.cuda.is_available():
    cnn = cnn.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

# 選擇損失函數

In [14]:
num_epochs = 10
LEARNING_RATE = 0.01

# 选择损失函数和优化方法
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=LEARNING_RATE, momentum=0.9)

# 将数据处理成Variable, 如果有GPU, 可以转成cuda形式
def get_variable(x):
    x = Variable(x)
    return x.cuda() if torch.cuda.is_available() else x

# 評估模型

In [ ]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    eval_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images = get_variable(images)
            labels = get_variable(labels)

            outputs = model(images)
            eval_loss += loss_func(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * correct / total
    eval_loss = eval_loss / total
#     print('Accuracy of the network on the test images: %d %%' % (
#         test_acc))
    return test_acc, eval_loss

# 訓練

In [ ]:
losses = []
accuracies = []
test_accuracies = []
# set the model to train mode initially
cnn.train()

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)


for epoch in range(num_epochs):
    since = time.time()
    running_loss = 0.0
    running_correct = 0.0
    for step, (images, labels) in enumerate(train_loader):
        images = get_variable(images)
        labels = get_variable(labels)

        cnn.train()
        outputs = cnn(images)
        loss = loss_func(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # calculate the loss/acc later
        running_loss += loss.item()
        running_correct += (labels==predicted).sum().item()
        if step % 100 == 0:
            print('Epoch: ', epoch, '| Step: ', step, '| loss: ', loss, '| predicted: ', predicted)

    epoch_duration = time.time()-since
    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100/BATCH_SIZE*running_correct/len(train_loader)
    print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
    
    losses.append(epoch_loss)
    accuracies.append(epoch_acc)

    # switch the model to eval mode to evaluate on test data
    cnn.eval()
    test_acc, test_loss = eval_model(cnn)
    print("eval_loss: %.4f, eval_acc: %.4f" % (test_loss, test_acc))
    test_accuracies.append(test_acc)
    
     # re-set the model to train mode after validating
    cnn.train()
    scheduler.step(test_acc)
    since = time.time()
    
print('Finished Training')


Epoch:  0 | Step:  0 | loss:  tensor(5.4952, device='cuda:0', grad_fn=<NllLossBackward>) | predicted:  tensor([194,  35], device='cuda:0')
Epoch:  0 | Step:  100 | loss:  tensor(9.1371, device='cuda:0', grad_fn=<NllLossBackward>) | predicted:  tensor([2, 2], device='cuda:0')
Epoch:  0 | Step:  200 | loss:  tensor(0.9784, device='cuda:0', grad_fn=<NllLossBackward>) | predicted:  tensor([1, 1], device='cuda:0')
Epoch:  0 | Step:  300 | loss:  tensor(1.4655, device='cuda:0', grad_fn=<NllLossBackward>) | predicted:  tensor([2, 2], device='cuda:0')
Epoch:  0 | Step:  400 | loss:  tensor(0.0877, device='cuda:0', grad_fn=<NllLossBackward>) | predicted:  tensor([2, 2], device='cuda:0')
Epoch:  0 | Step:  500 | loss:  tensor(1.2977, device='cuda:0', grad_fn=<NllLossBackward>) | predicted:  tensor([2, 2], device='cuda:0')
Epoch:  0 | Step:  600 | loss:  tensor(1.2281, device='cuda:0', grad_fn=<NllLossBackward>) | predicted:  tensor([2, 2], device='cuda:0')
Epoch:  0 | Step:  700 | loss:  tensor(

KeyboardInterrupt: 

In [ ]:
# Resize((512,512)
# Epoch 10, duration: 1372 s, loss: 0.4933, acc: 77.9643
# eval_loss: 0.1290, eval_acc: 76.2500

# 保存和加載整個網絡

In [ ]:
# 保存和加载整个模型, 包括: 网络结构, 模型参数等
torch.save(cnn, 'model.pkl')
cnn = torch.load('model.pkl')

# 保存和加載網絡中的參數
torch.save(cnn.state_dict(), 'params.pkl')
cnn.load_state_dict(torch.load('params.pkl'))